# TEST 

In [69]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import torch
from crossevopred.src.model.dummy_model import DummyModel
from crossevopred.src.model.dummy_trainer import DummyTrainer
from crossevopred.utils.data_processing_utils import *
from crossevopred.src.data.dataset import ExpressionDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
def window_array(arr, window_size):
    return np.array([np.mean(arr[i:i+window_size]) for i in range(0, len(arr)-window_size+1, window_size)])

# Simulate N sequences of length L. Each A +4, C+3, G+2, T+0 . 
# The output is a bed file with chr1 x y fasta_sequence [score_pos1, score_pos2, ...]
# in the simulation I want to have sequences that are very A rich and sequences that are very T rich.
# but still have some C and G.
def simulate_sequences(N, L, window_size, output_bed):
    sequences = []
    for i in range(N):
        sequence = np.random.choice(["A", "C", "G", "T"], L, p=[0.5, 0.25, 0.25, 0])
        sequences.append("".join(sequence))
    for i in range(N):
        sequence = np.random.choice(["A", "C", "G", "T"], L, p=[0, 0.25, 0.25, 0.5])
        sequences.append("".join(sequence))
    sequences = np.array(sequences)
    # scores are 4 for A, 3 for C, 2 for G, 0 for T
    scores = np.array([[4 if c == "A" else 3 if c == "C" else 2 if c == "G" else 0 for c in sequence] for sequence in sequences])
    scores = np.array([window_array(score, window_size) for score in scores])
    
    with open(output_bed, "w") as f:
        for i, (sequence, score) in enumerate(zip(sequences, scores)):
            f.write(f"chr1 {i*10} {i*10+L} {','.join(map(str, score))} {sequence}\n")


In [87]:
# Create simulated data
training_file = "../simulated/simulated.bed"
dataset_object = "../simulated/simulated_dataset.pt"
config_file = "../bin/crossevopred/config/test_config.yaml"
simulate_sequences(2, 10,5, output_bed=training_file)

In [88]:
# Create simulated dataset
dataset = ExpressionDataset(training_file)
torch.save(dataset, dataset_object)

In [89]:
# retrain model 
model   = DummyModel()
trainer = DummyTrainer()
trainining_datatset_loader = torch.load(dataset_object)
# Train the model
trainer.train(trainining_datatset_loader, config_file=config_file, model=model)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x40 and 4000x1)